In [1]:
# ATMS Python Implementation
# R. Shekhar
# CS152 - Lesson 8.1

import itertools

################ Data Type Definitions ###################

# Define a datatype to represent a logical literal
class Literal:
    def __init__(self,name='',description=''):
        self.name = name
        self.description = description

    def __repr__(self):
        return self.__str__()

    def __str__(self):
        return self.name

# Define a datatype to represent a horn clause
class HornClause:
    def __init__(self,conclusion,premise):
        self.premise = frozenset(premise)
        self.conclusion = conclusion

    def __eq__(self,other):
        return (self.premise == other.premise) and (self.conclusion == other.conclusion)

    def __hash__(self):
        return hash((self.premise,self.conclusion))

    def __repr__(self):
         return self.__str__()

    def __str__(self):
        if not self.premise:
            return self.conclusion.__str__()
        else:
            return self.conclusion.__str__() + u" \u21D0 " + u" \u2227 ".join("%s" % s.__str__() for s in self.premise)

In [9]:
################ Test Knowledge Base ###################

# Define the literals that will be used for inference
bronchitis = Literal('bronchitis')
influenza = Literal('influenza')
smokes = Literal('smokes')
coughing = Literal('coughing')
wheezing = Literal('wheezing')
fever = Literal('fever')
infection = Literal('infection')
sore_throat = Literal('sore_throat')
nonsmoker = Literal('nonsmoker')
dyspnea = Literal('dyspnea')
asthma = Literal('asthma')
false = Literal('false')

on_t1 = Literal('on_t1')
on_t2 = Literal('on_t2')
on_t3 = Literal('on_t3')
off_t1 = Literal('off_t1')
off_t2 = Literal('off_t2')
off_t3 = Literal('off_t3')
pressurized_p1 = Literal('pressurized_p1')
pressurized_p2 = Literal('pressurized_p2')
pressurized_p3 = Literal('pressurized_p3')
plugged_sink = Literal('plugged_sink')
plugged_bath = Literal('plugged_bath')
unplugged_sink = Literal('unplugged_sink')
unplugged_bath = Literal('unplugged_bath')
flow_sink = Literal('flow_sink')
flow_bath = Literal('flow_bath')
wet_sink = Literal('wet_sink')
wet_bath = Literal('wet_bath')
blocked_d1 = Literal('blocked_d1')
blocked_d2 = Literal('blocked_d2')
blocked_d3 = Literal('blocked_d3')
clear_d1 = Literal('clear_d1') 
clear_d2 = Literal('clear_d2')
clear_d3 = Literal('clear_d3')
flow_d1 = Literal('flow_d1') 
flow_d2 = Literal('flow_d2')
flow_d3 = Literal('flow_d3')
leaky_t1 = Literal('leaky_t1')
leaky_t2 = Literal('leaky_t2') 
leaky_t3 = Literal('leaky_t3')


# Define the assumables
assumables_set = {influenza,infection,smokes,nonsmoker}

# Define the KB (atomic clauses can be represented with an empty premise)
KB = {HornClause(bronchitis,[influenza]),
      HornClause(bronchitis,[smokes]),
      HornClause(coughing,[bronchitis]),
      HornClause(wheezing,[bronchitis]),
      HornClause(fever,[influenza]),
      HornClause(fever,[infection]),
      HornClause(sore_throat,[influenza]),
      HornClause(false,[smokes,nonsmoker]),
      HornClause(dyspnea,[influenza,smokes]),
      HornClause(fever,[]), # Atomic clause
      }

KB_new = {HornClause(pressurized_p2,[on_t1,pressurized_p1]),
      HornClause(flow_bath,[on_t2,pressurized_p2]),
      HornClause(wet_bath,[flow_bath]),
      HornClause(flow_d2,[wet_bath,unplugged_bath,clear_d2,clear_d1]),
      HornClause(flow_d1 ,[flow_d2]),
      HornClause(pressurized_p3,[on_t1,pressurized_p1]),
      HornClause(flow_sink,[on_t3,pressurized_p3]),
      HornClause(wet_sink,[flow_sink]),
      HornClause(flow_d3,[wet_sink,unplugged_sink,clear_d3,clear_d1]),
      HornClause(flow_d1,[flow_d3]),
      HornClause(unplugged_sink,[]),
      HornClause(pressurized_p1,[]), # Atomic clause
      }

In [10]:
################ ATMS Implementation ###################

# store the goal clauses in a separate set
goal_clauses = {c for c in KB if c.conclusion == false}
# Duplicate the KB for later manipulation
all_clauses = KB.copy()

# Define the assumables list that will be updated on each iteration of ATMS
C = {a:{frozenset([a])} for a in assumables_set}

# Iterate until we reach a fixed point (cannot infer anything more)
while True:
    # Find all the clauses we can infer (i.e. those whose premise are in the list of inferred literals C)
    inferred_clauses = {c for c in all_clauses if c.premise.issubset(C)}
    # Exit if we cannot infer anything more
    if not inferred_clauses: break
    # Loop over all clauses inferred those symbols inferred up to this point
    for clause in inferred_clauses:
            # Remove the clause from the list of all clauses
            all_clauses.remove(clause)
            # If this is the first time the conclusion has been drawn, initialize it.
            if clause.conclusion not in C: C[clause.conclusion] = set()
            # If the premise of the clause is empty, we don't need to find any combination of assumable sets
            if clause.premise:
                assumable_set = {frozenset(C[u]) for u in clause.premise}
                assumable_combos = {frozenset.union(*u) for u in itertools.product(*assumable_set)}
            else: assumable_combos = {frozenset()}

            # Optimizations
            # 1. Check if any of the assumables superset those already in the list itself or already recorded for this literal and remove them
            assumable_combos = {a for a in assumable_combos if not any (a > b for b in assumable_combos)}
            assumable_combos = {a for a in assumable_combos if not any (a > b for b in C[clause.conclusion])}
            # 2. Check if any subset has already been falsified by the goal clauses, and remove them
            assumable_combos = {a for a in assumable_combos if clause.conclusion is false or not any (a >= b.premise for b in goal_clauses)}
            # 3. Check if any of the assumables sets are a subset those already on the list and modify the list (a more minimal explanation has been found, so can replace the one already there)
            C[clause.conclusion].difference_update({a for a in C[clause.conclusion] if any (a > b for b in assumable_combos)})

            if assumable_combos: C[clause.conclusion].update(assumable_combos)

# Print out the minimal explanations for each clause
print("====ATMS Results====")
print("Assumables: " + str(assumables_set) + "\n")
print("KB: " + str(KB) + "\n")
given_literals = {c for c in KB if not c.premise}
if given_literals: print("Literals given directly from KB: " + str(given_literals) + "\n")
explanable_literals = [c for c in C if c != false and c not in assumables_set and C[c]]
max_len = max([len(l.name) for l in explanable_literals])
print("-------Minimal Explanations-------")

if false in C: max_len = max(max_len,len("conflicts"))
for c in explanable_literals:
    print("{1:<{0}}: {2}".format(max_len,c.name,", ".join(["{" + u" \u2227 ".join([str(x) for x in t]) + "}" for t in C[c]]) + "."))

if false in C: print("\n{1:<{0}}: {2}".format(max_len,"conflicts", ",".join(["{" + u" \u2227 ".join([str(x) for x in t]) + "}" for t in C[false]]) + "."))

====ATMS Results====
Assumables: {nonsmoker, infection, smokes, influenza}

KB: {dyspnea ⇐ smokes ∧ influenza, bronchitis ⇐ smokes, coughing ⇐ bronchitis, fever ⇐ influenza, sore_throat ⇐ influenza, wheezing ⇐ bronchitis, false ⇐ nonsmoker ∧ smokes, fever, bronchitis ⇐ influenza, fever ⇐ infection}

Literals given directly from KB: {fever}

-------Minimal Explanations-------
dyspnea    : {smokes ∧ influenza}.
bronchitis : {smokes}, {influenza}.
fever      : {}.
sore_throat: {influenza}.
coughing   : {smokes}, {influenza}.
wheezing   : {smokes}, {influenza}.

conflicts  : {nonsmoker ∧ smokes}.
